In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, transforms
from torch.utils.data import DataLoader, Subset

In [2]:
transform = transforms.Compose([
        transforms.ToTensor(),
        transforms.Normalize((0.1307, ), (0.3081, ))
])

train_dataset = datasets.MNIST(root = "./data", train = True, download = True, transform = transform)
test_dataset = datasets.MNIST(root = "./data", train = False, download = True, transform = transform)

In [3]:
train_subset = Subset(train_dataset, range(200))
test_subset = Subset(test_dataset, range(50))

train_loader = DataLoader(train_subset, batch_size = 10, shuffle = True)
test_loader = DataLoader(test_subset, batch_size = 10, shuffle = False)

In [4]:
class SimpleANN(nn.Module):
        def __init__(self):
                super(SimpleANN, self).__init__()
                self.fc1 = nn.Linear(28 * 28, 128)
                self.fc2 = nn.Linear(128, 64)
                self.fc3 = nn.Linear(64, 10)
        
        def forward(self, x):
                x = torch.flatten(x, start_dim = 1)
                x = torch.relu(self.fc1(x))
                x = torch.relu(self.fc2(x))
                x = self.fc3(x)
                return x
        
model = SimpleANN()
optimizer = optim.Adam(model.parameters(), lr = 0.001)
criterion = nn.CrossEntropyLoss()

In [5]:
def train_model(num_epochs):
        for epoch in range(num_epochs):
                model.train()
                train_loss = 0
                correct_train = 0
                total_train = 0
                for data, target in train_loader:
                        optimizer.zero_grad()
                        op = model(data)
                        loss = criterion(op, target)
                        loss.backward()
                        optimizer.step()
                        train_loss += loss.item()
                        predicted = torch.argmax(op.data, dim = 1)
                        total_train += target.size(0)
                        correct_train += (predicted == target).sum().item()

                avg_train_loss = train_loss / len(train_loader)
                train_acc = 100 * correct_train / total_train

                model.eval()
                test_loss = 0
                correct_test = 0
                total_test = 0

                with torch.no_grad():
                        for data, target in test_loader:
                                op = model(data)
                                loss = criterion(op, target)
                                test_loss += loss.item()
                                predicted = torch.argmax(op.data, dim = 1)
                                total_test += target.size(0)
                                correct_test += (predicted == target).sum().item()
                
                avg_test_loss = test_loss / len(test_loader)
                test_acc = 100 * correct_test / total_test

                print(f'Epoch {epoch + 1}, Train Loss: {avg_train_loss: .4f}, Train Accuracy: {train_acc:.8f}%,'
                      f'Test Loss: {avg_test_loss: .4f}, Test Accuracy: {test_acc: .8f}%')
                

train_model(10)


Epoch 1, Train Loss:  2.0127, Train Accuracy: 37.00000000%,Test Loss:  1.6204, Test Accuracy:  62.00000000%
Epoch 2, Train Loss:  1.0306, Train Accuracy: 74.50000000%,Test Loss:  1.0324, Test Accuracy:  74.00000000%
Epoch 3, Train Loss:  0.5267, Train Accuracy: 87.50000000%,Test Loss:  0.8695, Test Accuracy:  72.00000000%
Epoch 4, Train Loss:  0.3117, Train Accuracy: 94.00000000%,Test Loss:  0.7235, Test Accuracy:  74.00000000%
Epoch 5, Train Loss:  0.1721, Train Accuracy: 98.00000000%,Test Loss:  0.5980, Test Accuracy:  84.00000000%
Epoch 6, Train Loss:  0.0993, Train Accuracy: 98.50000000%,Test Loss:  0.5848, Test Accuracy:  80.00000000%
Epoch 7, Train Loss:  0.0610, Train Accuracy: 100.00000000%,Test Loss:  0.5513, Test Accuracy:  82.00000000%
Epoch 8, Train Loss:  0.0388, Train Accuracy: 100.00000000%,Test Loss:  0.5452, Test Accuracy:  78.00000000%
Epoch 9, Train Loss:  0.0273, Train Accuracy: 100.00000000%,Test Loss:  0.5443, Test Accuracy:  80.00000000%
Epoch 10, Train Loss:  0.